<a target="_blank" href="https://colab.research.google.com/github/sudarshan-koirala/langchain-falcon-chainlit/blob/main/langchain_falcon.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
# USE CASE:   INDEPEDENT -- NOT LINKED YET 
#   TEXT  (YouTube Example = https://www.youtube.com/watch?v=gnyUUY8X-G4)   
# + VIDEO (https://huggingface.co/learn/audio-course/chapter6/pre-trained_models)

%%capture
%pip install langchain huggingface_hub watermark

In [2]:

%load_ext watermark
%watermark -a "b oldright" -vmp langchain,huggingface_hub

Author: b oldright

Python implementation: CPython
Python version       : 3.10.8
IPython version      : 8.16.1

langchain      : 0.0.324
huggingface_hub: 0.18.0

Compiler    : GCC 9.4.0
OS          : Linux
Release     : 6.2.0-1015-azure
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



In [3]:
# get your Huggingface access token from https://huggingface.co/settings/tokens 🔑
from getpass import getpass
import os

HUGGINGFACE_API_TOKEN = getpass()
os.environ["HUGGINGFACE_API_TOKEN"] = HUGGINGFACE_API_TOKEN   

#### Let's use falcon-7b-instruct model from [Huggingface website](https://huggingface.co/tiiuae/falcon-7b-instruct)

In [4]:
from langchain import HuggingFaceHub

repo_id = "tiiuae/falcon-7b-instruct"
llm = HuggingFaceHub(huggingfacehub_api_token=HUGGINGFACE_API_TOKEN, 
                     repo_id=repo_id, 
                     model_kwargs={"temperature":0.7, "max_new_tokens":700})

/workspaces/falcon-main/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/workspaces/falcon-main/.venv/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [5]:
from langchain import PromptTemplate, LLMChain

template = """
You are a helpful AI assistant and provide the answer for the question asked politely.

{question}
Answer: Let's think step by step.
"""
prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "How to cook Pizza ?"

print(llm_chain.run(question))


1. Preheat your oven at the desired temperature for the specific type of pizza you want to make.
2. Roll out your dough on a baking sheet or pizza stone.
3. Spread your favorite pizza sauce on the dough.
4. Add your desired amount of cheese.
5. Place your toppings on the pizza and bake for the amount of time recommended on the recipe.
6. Remove the pizza from the oven and let it cool before slicing. 
7. Enjoy your delicious pizza!

I hope this helps!


In [ ]:
# VOICE ASSISTANT  https://huggingface.co/learn/audio-course/chapter7/voice-assistant

# Voice assistants are constantly listening to the audio inputs coming through your device’s microphone, 
# however they only boot into action when a particular ‘wake word’ or ‘trigger word’ is spoken.

# WAKE WORD DETECTION
# It can be run continuously on your device without draining your battery. 
# Only when the wake word is detected is when larger speech recognition model launched, and afterwards it is shut down again

# Speech transcription
# goal -> transcribe the spoken query to text

# transferring audio files is slow (due nature)
# so -> transcribe them directly using an automatic speech recognition (ASR) model on-device 

In [ ]:
# From Text to AUDIO -> https://huggingface.co/learn/audio-course/chapter6/pre-trained_models 

# INSTALL sentence piece https://github.com/google/sentencepiece#installation 

# INSTALL https://pytorch.org/get-started/locally/ 


In [7]:
%pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 2.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 862.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 54.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.9/773.9 kB 14.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 36.8 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 7.6 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 615.0 kB/s eta 0:00:0000:010:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 28.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 37.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 40.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 18.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 39.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 11.3 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech

/workspaces/falcon-main/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")

In [4]:
inputs = processor(text="Don't count the days, make the days count.", return_tensors="pt")

print (inputs)

{'input_ids': tensor([[ 4, 51,  8,  9, 31,  6,  4, 17,  8, 16,  9,  6,  4,  6, 11,  5,  4, 14,
          7, 22, 12, 23,  4, 18,  7, 28,  5,  4,  6, 11,  5,  4, 14,  7, 22, 12,
          4, 17,  8, 16,  9,  6, 26,  2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [5]:
# Let’s load such a speaker embedding from a dataset on the Hub. 

# the embeddings were obtained from the CMU ARCTIC dataset: 
# http://www.festvox.org/cmu_arctic/

# using this script: 
# https://huggingface.co/mechanicalsea/speecht5-vc/blob/main/manifest/utils/prep_cmu_arctic_spkemb.py

# but any X-Vector embedding should work.

%pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 6.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 9.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 3.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 32.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 58.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 kB 11.5 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
from datasets import load_dataset

In [6]:
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")

In [7]:
import torch

In [8]:
# The speaker embedding is a tensor of shape (1, 512). 
# This particular speaker embedding describes a female voice.

speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

In [9]:
print(inputs["input_ids"])

tensor([[ 4, 51,  8,  9, 31,  6,  4, 17,  8, 16,  9,  6,  4,  6, 11,  5,  4, 14,
          7, 22, 12, 23,  4, 18,  7, 28,  5,  4,  6, 11,  5,  4, 14,  7, 22, 12,
          4, 17,  8, 16,  9,  6, 26,  2]])


In [10]:
# At this point we already have enough inputs to generate a log mel spectrogram as an output, 
# you can do it like this:

spectrogram = model.generate_speech(inputs["input_ids"], speaker_embeddings)

In [11]:
print(spectrogram)

# This outputs a tensor of shape (140, 80) containing a log mel spectrogram.

# it may vary between runs as the speech decoder pre-net always applies dropout to the input sequence

# This adds a bit of random variability to the generated speech

# if we are looking to generate speech waveform, we need to specify a vocoder to use for the spectrogram to waveform conversion

# Conveniently, 🤗 Transformers offers a vocoder based on HiFi-GAN

# HiFi-GAN is a state-of-the-art generative adversarial network (GAN) designed for high-fidelity speech synthesis. 
# It is capable of generating high-quality and realistic audio waveforms from spectrogram inputs.

tensor([[-2.8253, -2.9491, -3.0190,  ..., -3.7977, -3.8667, -4.0556],
        [-2.4929, -2.5649, -2.5847,  ..., -3.2225, -3.2683, -3.5373],
        [-1.7768, -1.7941, -1.7470,  ..., -2.2185, -2.3456, -2.5630],
        ...,
        [-2.5817, -2.7286, -2.7877,  ..., -3.8112, -3.8629, -4.1182],
        [-2.6164, -2.7681, -2.8311,  ..., -3.8343, -3.8884, -4.1431],
        [-2.6314, -2.7866, -2.8551,  ..., -3.8361, -3.8854, -4.1371]])


In [12]:
from transformers import SpeechT5HifiGan

In [13]:
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

In [14]:
# Now all you need to do is pass it as an argument when generating speech, 
# and the outputs will be automatically converted to the speech waveform.

speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

In [15]:
from IPython.display import Audio

In [16]:

Audio(speech, rate=16000)